<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ASR-Privacy/1-second-speech-predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'text_normalized', 'text_original', 'speaker_id', 'path', 'chapter_id', 'id'],
        num_rows: 5736
    })
})

In [7]:
ds['train']['audio'][0]

{'path': '2902_9008_000002_000000.wav',
 'array': array([-3.05175781e-05, -3.05175781e-04, -3.96728516e-04, ...,
         8.54492188e-04,  8.54492188e-04,  1.40380859e-03]),
 'sampling_rate': 24000}

In [ ]:
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
ds = load_dataset("HamdanXI/libritts_dev_dataset")

# Parameters
sample_rate = 24000  # Assuming the sampling rate from your screenshot
segment_duration = 1  # Duration of audio segment in seconds
segment_length = sample_rate * segment_duration  # Number of samples in the segment

# Preprocess the dataset
def process_audio(batch):
    waveform = batch["audio"]["array"]
    # Ensure we take only the first `segment_duration` seconds
    if len(waveform) >= segment_length:
        waveform = waveform[:segment_length]
    else:
        # Pad if shorter than required length
        padding = segment_length - len(waveform)
        waveform = np.pad(waveform, (0, padding), mode="constant")
    # Flatten waveform to use as features
    batch["features"] = waveform
    return batch

# Apply processing
ds = ds.map(process_audio, remove_columns=["audio"])

# Extract features and labels
X = np.array([item["features"] for item in ds["train"]])
y = np.array([item["speaker_id"] for item in ds["train"]])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
ds = load_dataset("HamdanXI/libritts_dev_dataset")

# Parameters
sample_rate = 16000  # Wav2Vec2 expects audio at 16kHz
segment_duration = 1  # 1-second audio segments
segment_length = sample_rate * segment_duration

# Create a mapping from string speaker IDs to integers
unique_labels = sorted(set(ds["train"]["speaker_id"]))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Load Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=len(unique_labels)
)
model.config.label2id = label2id
model.config.id2label = id2label

# Preprocess the dataset
def preprocess_audio(batch):
    waveform = torch.tensor(batch["audio"]["array"], dtype=torch.float32)  # Ensure Float32 type
    # Ensure 1-second segments
    if len(waveform) >= segment_length:
        waveform = waveform[:segment_length]
    else:
        padding = segment_length - len(waveform)
        waveform = torch.nn.functional.pad(waveform, (0, padding))
    # Resample to 16kHz
    if batch["audio"]["sampling_rate"] != sample_rate:
        resampler = torchaudio.transforms.Resample(
            orig_freq=batch["audio"]["sampling_rate"], new_freq=sample_rate
        )
        waveform = resampler(waveform)
    # Normalize the waveform
    waveform = (waveform - waveform.mean()) / waveform.std()
    return {
        "input_values": processor(waveform.numpy(), sampling_rate=sample_rate).input_values[0],
        "label": label2id[batch["speaker_id"]],  # Convert label to integer
    }

# Apply preprocessing
ds = ds.map(preprocess_audio, remove_columns=["audio"])

# Split dataset into train/test
train_test_split = ds["train"].train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

# Define data collator
def data_collator(features):
    input_values = torch.tensor([f["input_values"] for f in features], dtype=torch.float32)
    labels = torch.tensor([f["label"] for f in features], dtype=torch.long)  # Use long for classification labels
    return {"input_values": input_values, "labels": labels}

# Training arguments
training_args = TrainingArguments(
    output_dir="./wav2vec2-speaker-id",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    warmup_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(f"Metrics: {metrics}")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5736 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-06bd6ae605e9>:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,7.048200,3.515486


Epoch,Training Loss,Validation Loss
1,7.048200,3.515486
